In [2]:
import os
import SimpleITK as sitk
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

import time

In [3]:
import sys
sys.path.insert(1, '/workspace')

In [4]:
from_dir = '/workspace/BoneMeta_all_192/images'
to_dir = '/workspace/BoneMeta_all_192/images_bone'
seg_dir = '/workspace/BoneMeta_all_192/preds_seg'

In [13]:
from multiprocessing import Pool

In [16]:
class BoneSegmentor:
    def __init__(self, args):
        self.from_dir, self.to_dir, self.seg_dir = args
    
    def save_np(self, np_image, file_name):
        np_image = np_image.astype(np.int16)
        np.save(os.path.join(self.to_dir, file_name), np_image)
        
    def segment_bone(self, file_name):
        image = np.load(os.path.join(from_dir, file_name))
        bone_seg = np.load(os.path.join(seg_dir, file_name))
        bone_image = image * bone_seg - 1024 * (1-bone_seg)
        return bone_image
    
    def segment_and_save(self, file_name):
        bone_image = self.segment_bone(file_name)
        self.save_np(bone_image, file_name)
        
    def segment_all(self):
        os.makedirs(self.to_dir, exist_ok=True)

        t = time.perf_counter()

        file_list = os.listdir(self.from_dir)
        pool = Pool(processes=32)
        pool.map(self.segment_and_save, file_list)
        pool.close()
        pool.join()

        elapsed_time = time.perf_counter() - t

        print(f'''
        consumed_time: {elapsed_time}s
        ''')

In [17]:
segmentor = BoneSegmentor((from_dir, to_dir, seg_dir))

In [18]:
segmentor.segment_all()


        consumed_time: 5.78821772900119s
        
